In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

$\bf{\sigma_z}$ as reference basis

In [34]:
n_vis = 3  # Number of visible neurons.
n_hid = 2  # Number of hidden neurons.
ratio = n_hid / n_vis  # Expressive power.

W = np.random.random((n_hid, n_vis))  # Weights matrix.
b = np.random.random(n_vis)  # Biases to visible neurons.
c = np.random.random(n_hid)  # Biases to hidden neurons.

params = {
    "lambda": {
        "W": np.random.random((n_hid, n_vis)),
        "b": np.random.random(n_vis),
        "c": np.random.random(n_hid)
    },

    "mu": {
        "W": np.random.random((n_hid, n_vis)),
        "b": np.random.random(n_vis),
        "c": np.random.random(n_hid)
    }
}

print(ratio)
params

0.6666666666666666


{'lambda': {'W': array([[ 0.12995647,  0.60859445,  0.03596888],
         [ 0.78243525,  0.10541418,  0.28989045]]),
  'b': array([ 0.71262189,  0.13512335,  0.56960176]),
  'c': array([ 0.7407072 ,  0.58162135])},
 'mu': {'W': array([[ 0.37604412,  0.51080663,  0.41208797],
         [ 0.76828467,  0.23351423,  0.14828215]]),
  'b': array([ 0.84203523,  0.82530931,  0.06916255]),
  'c': array([ 0.85793757,  0.67202164])}}

In [35]:
def boltzmann_joint_distribution(vis, hid, weights, b_bias, c_bias):
    """
    (6) formula from original paper.
    """
    tot = hid.dot(weights).dot(vis)
    tot += vis.dot(b_bias)
    tot += hid.dot(c_bias)
    return np.exp(tot)

In [39]:
vis = np.array([1,0,0])
hid = np.array([0,1])

boltzmann_joint_distribution(vis, hid, W, b, c)

3.2647848363614869

In [37]:
# All possible variants of bit arrays.
import itertools


def get_ideal_state(n=n_vis):
    ideal_state = np.ones(n) / np.sqrt(n)
    return ideal_state


def get_all_states(n=n_vis):
    all_states = np.array(list(map(np.array, itertools.product([0, 1], repeat=n))))[1:]
    all_states = np.array([np.sqrt(x / x.sum()) for x in all_states])
    return all_states

In [6]:
def fidelity(state1, state2):
    return state1.dot(state2) ** 2

**deprecated**

We decided to sample $a_1, a_2, \dots, a_n$ to build state $|\psi_i>=a_1 |10\dots0> + \ldots + a_n |0\dots01>$ such that $a_1^2 + \ldots + a_n^2 = 1$

In [7]:
def sample_a(size, n=n_vis): 
#     res = []
#     for i in range(size):
#         tmp = np.random.random(n)
#         tmp = tmp / tmp.sum()
#         tmp = np.array([np.sqrt(x) for x in tmp])
#         res.append(tmp)
    
    res = np.random.choice([0, 1], (size, n))
    res = np.array(list(filter(lambda x: x.sum() > 0, res)))
    
    tmp = np.sqrt(np.diag(res.dot(res.T)))
    tmp = tmp.reshape(len(tmp), -1)
    
    res = res / tmp
        
    return res

In [71]:
sample_a(10)

array([[ 0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678],
       [ 0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ],
       [ 0.70710678,  0.        ,  0.70710678],
       [ 0.        ,  1.        ,  0.        ]])

In [26]:
np.random.choice([0, 1], (10, 3))

array([[0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 0]])

In [17]:
def sample_dataset(size, dimens=n_vis, num_samples=1000, num_of_bins=20, verbose=False):
    """
    size # Dataset size.
    num_samples = 1000  # Number of sampled sets of `a`.
    num_of_bins = 20
    
    """
    a_samples = sample_a(num_samples, dimens)
    num_samples = len(a_samples)

    fidelities = np.array([fidelity(x, ideal_state) for x in a_samples])

    probs = fidelities / sum(fidelities)
    
    if verbose:
        plt.scatter(np.arange(num_samples), sorted(fidelities))
        plt.title('Fidelity of sampled states - set of $\overrightarrow{a}$')
        plt.xlabel('Sample')
        plt.show()
        
        plt.hist(fidelities, bins=20)
        plt.title("Distribution of fidelities")
        plt.show()

        plt.scatter(np.arange(num_samples), sorted(probs))
        plt.title('Probability - fidelities / sum(fidelities)')
        plt.show()
        
    eps = 1e-7
    bins = np.linspace((1 - eps) * fidelities.min(), (1 + eps) * fidelities.max(), num_of_bins + 1)
    bins_indices = np.digitize(fidelities, bins)
    
#     if verbose:
#         plt.title('Distribution of bins indices')
#         plt.hist(bins_indices)
#         plt.show()
        
    uniq_vals = np.arange(1, num_of_bins + 1)
    probs = np.array([(bins_indices == num).sum() for num in uniq_vals]) / num_samples
    
    if verbose:
        plt.title('Probability - after binarization')
        plt.plot(probs, '.')
        plt.show()
        
    bins_sampled = np.random.choice(uniq_vals, p=probs, size=size)
    
    tmp = list(enumerate(bins_indices))

    bins_states = dict()
    for num in set(bins_indices):
        bins_states[num] = np.array([i for i, x in tmp if x == num])
        
    res_a_sampled = []
    for num in bins_sampled:
        if num in bins_states:
            res_a_sampled.append(a_samples[np.random.choice(bins_states[num])])
    
    res_a_sampled = np.array(res_a_sampled)
            
    if verbose:
        plt.hist([fidelity(x, ideal_state) for x in res_a_sampled], bins=20)
        plt.title("Distribution of fidelities - dataset")
        plt.show()
        
    return res_a_sampled

In [84]:
dataset1 = sample_dataset(500, num_of_bins=3, verbose=False)

In [162]:
def boltzmann_margin_distribution(vis, weights, b_bias, c_bias):
    """
    (7) formula from original paper.
    """
    tmp = weights.dot(vis)
    tmp += c_bias
    tmp = np.exp(tmp)
    tmp += 1
    tmp = np.log(tmp)
    tmp = tmp.sum()
    tmp += vis.dot(b_bias)
    return np.exp(tmp)

boltzmann_margin_distribution(vis, W, b, c)

23.9722352336107

In [171]:
def p_k(k, vis, params):
    """
    `k` is either "lambda" or "mu".
    """
    return boltzmann_margin_distribution(vis, params[k]['W'],
                                         params[k]['b'], params[k]['c'])

def p_lambda(vis, params):
    return p_k('lambda', vis, params)

def p_mu(vis, params):
    return p_k('mu', vis, params)

def phi_mu(vis, params):
    return np.log(p_mu(vis, params))

In [172]:
# All possible variants of bit arrays.
# import itertools
# n = 3
# lst = np.array(list(map(np.array, itertools.product([0, 1], repeat=n))))

sigmas = np.eye(n_vis)

# Normalization constant.
Z_lambda = sum(list(map(lambda x: p_lambda(x), sigmas)))
Z_lambda

33.851975048683016

In [175]:
def psi_lambda_mu(vis):
    """
    (8) formula from original paper.
    """
    tmp = 1j * phi_mu(vis) / 2
    tmp = np.exp(tmp)
    tmp *= np.sqrt(p_lambda(vis) / Z_lambda)
    return tmp

In [176]:
psi_lambda_mu(vis)

(0.1863799785817617+0.62303563240778093j)

In [754]:
def quasi_prob(vis, u=None):
    """
    (12) formula from original paper.
    """
    tmp = 1j * phi_mu(vis) / 2
    tmp = np.exp(tmp)
    tmp *= np.sqrt(p_lambda(vis))
    
    if u:
        return u * tmp
    else:
        return tmp

In [755]:
def D_k_sigma(k, sigma):
    """
    sigma (np.array)
    
    """
    weights = params[k]['W']
    c_bias = params[k]['c']
    
    grad_b = sigma
    
    tmp = weights.dot(sigma)
    tmp += c_bias
    tmp = np.exp(tmp)
    grad_c = tmp / (1 + tmp)
    
    grad_W = np.outer(grad_c, sigma)
    
    res = {
        'W': grad_W,
        'c': grad_c,
        'b': grad_b
    }
    
    return res

In [756]:
D_k_sigma('lambda', vis)

{'W': array([[ 0.63585126,  0.        ,  0.        ],
        [ 0.51863706,  0.        ,  0.        ]]),
 'b': array([1, 0, 0]),
 'c': array([ 0.63585126,  0.51863706])}

In [771]:
def averaged_D_k_Qb(k, batch):
    """
    (15) formula from original paper.
    
    """
    res_W = None
    res_b = None
    res_c = None
    quasi_probs = None
    
    for x in batch:
        gradients = D_k_sigma(k, x)
        quasi = quasi_prob(x)
        
        if quasi_probs is None:
            quasi_probs = quasi
            res_W = quasi * gradients["W"]
            res_b = quasi * gradients["b"]
            res_c = quasi * gradients["c"]
        else:
            quasi_probs += quasi
            res_W += quasi * gradients["W"]
            res_b += quasi * gradients["b"]
            res_c += quasi * gradients["c"]
    
    res_W /= quasi_probs
    res_b /= quasi_probs
    res_c /= quasi_probs
    
    res = {
        'W': res_W,
        'c': res_c,
        'b': res_b
    }
    
    return res

In [782]:
tmp = averaged_D_k_Qb('lambda', dataset1)['W']

In [757]:
def averaged_D_lambda_p_lambda(batch):
    """
    (17) formula from original paper.
    
    """
    n = len(batch)
    
    res_W = None
    res_b = None
    res_c = None
    
    for x in batch:
        gradients = D_k_sigma('lambda', x)
        
        if res_W is None:
            res_W = gradients["W"]
            res_b = gradients["b"]
            res_c = gradients["c"]
        else:
            res_W += gradients["W"]
            res_b += gradients["b"]
            res_c += gradients["c"]
            
    res_W /= n
    res_b /= n
    res_c /= n
    
    res = {
        'W': res_W,
        'c': res_c,
        'b': res_b
    }
    
    return res

In [758]:
len(dataset1)

500

In [764]:
averaged_D_lambda_p_lambda(dataset1[:5])

{'W': array([[ 0.38757272,  0.3851625 ,  0.44103962],
        [ 0.33641119,  0.33435325,  0.38344697]]),
 'b': array([ 0.53687518,  0.53333991,  0.60643279]),
 'c': array([ 0.7243129,  0.6291669])}

In [ ]:
def gradient_lambda_div(dataset, N_b=1):
    assert len(dataset) == N_b, "Number of bases must be equal to length of density measurements dataset"
    
    for b in range(N_b):
        len(dataset[b])
        tmp = 0
        for sigma in dataset[b]:
            tmp += averaged_D_k_Qb('lambda', sigma)

Using defined distribution we can assign probability to every outcome and after that we can sample some configuration $\bf{\sigma}$.

Better: perform Gibbs sampling as in usual RBM using two conditional distributions $p_{\lambda}(\sigma\vert h)$ and $p_{\lambda}(h\vert \sigma)$.

Series of data sets $D_b$ for each base $b=1,\dots, N_b - 1$

$U_b(\sigma, \sigma^{[b]})$ - basis transformation matrix

In [177]:
N_b = 2 # Number of bases.
dataset = dict()

In [178]:
dataset[0] = np.array([[1,0,0],
                       [0,0,1]])

dataset[1] = np.array([[1,0,0]])

assert len(dataset) == N_b, "Number of bases must be equal to size of dataset"

In [139]:
def total_divergence():
    tot = N_b * np.log(Z_lambda)
    
    return tot

In [140]:
total_divergence()

9.4343779095210216